In [5]:
import requests
import json
import pandas as pd

from tqdm.auto import tqdm

In [6]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-12 06:41:26--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-12 06:41:26 (58.4 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [46]:
with open('document-with-ids.json', 'r') as file:
    documents = json.load(file)

In [47]:
index = minsearch.Index(
    text_fields=["question", "answer", "category"],
    keyword_fields=["section"]
)

index.fit(documents)

In [48]:
def search(query):
    boost = {'question': 2.0, 'category': 0.2}

    results = index.search(
        query=query,
        filter_dict={'section': 'general claim benefits'},
        boost_dict=boost,
        num_results=10
    )

    return results

In [49]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

Before running the codes for huggingfaces to download llm models and tokenizer, check and change the location of the download with the codes below. On colab, change the runtime hardware type to T4 GPU

In [13]:
#check its running on gpu
!nvidia-smi

Thu Sep 12 06:44:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
#Checks the size of the disk spaces
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   33G   80G  30% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G     0  5.7G   0% /dev/shm
/dev/root       2.0G  1.2G  820M  59% /usr/sbin/docker-init
tmpfs           6.4G  1.1M  6.4G   1% /var/colab
/dev/sda1        86G   54G   33G  63% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


changes the download location

import os

os.environ['HF_HOME'] = '/run/cache/'

In [15]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.1 MB/s eta 0:00:00


In [16]:
!pip install transformers

In [17]:
!pip install accelerate

Go to huggingfaces and copy usage of Flan/T5-XL then run the code as shown below. This imports the model and a tokenizer for google flan/t5-xl

In [18]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [19]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [50]:
documents[0]

{'category': 'Manage existing benefit',
 'question': 'How do I update my benefit information?',
 'answer': 'You can update your benefit information online through your account.',
 'section': 'general claim benefits',
 'id': '30eada08-5708-5c5c-9df8-0f7d5d4dc131'}

In [51]:
def build_prompt(query, search_results):
    prompt_template = """
You are an expert in United Kingdom Benefit Claims and Medical Negligence Claims. Answer the QUESTION based on the CONTEXT from
the FAQ databases of Benefits database and NHS claims management.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        if 'answer' in doc and doc['answer']:
            context += f"Category: {doc['category']}\nQuestion: {doc['question']}\nAnswer: {doc['answer']}\nSection: {doc['section']}\n\n"

    if not context.strip():
        context = "No relevant context found for this question."

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

def llm(prompt):
    input_text = "translate English to German: How old are you?"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids)
    result = tokenizer.decode(outputs[0])
    return result

In [52]:
rag("How do I update my benefit information?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'<pad>You can appeal a benefit decision by requesting a mandatory reconsideration within one month.'

In [53]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 200),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [63]:
def rag(query: dict) -> str:
    # Perform the search using minsearch
    search_results = search(query['question'])

    # Build the prompt based on the search results
    prompt = build_prompt(query['question'], search_results)

    # Call the LLM (Flan-T5) to generate the answer
    answer = llm(prompt)  # No need for model parameter, since Flan-T5 is already set in `llm`

    return answer

In [64]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [65]:
ground_truth[0]

{'question': 'How can I change my existing benefit details?',
 'section': 'general claim benefits',
 'document': '30eada08-5708-5c5c-9df8-0f7d5d4dc131'}

In [66]:
doc_idx = {d['id']: d for d in documents}

In [67]:
answers = {}

In [ ]:
for i, rec in enumerate(tqdm(ground_truth)):
    if i in answers:
        continue

    answer_llm = rag(rec)
    doc_id = rec['document']
    original_doc = doc_idx[doc_id]
    answer_orig = original_doc['answer']

    answers[i] = {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['question'],
        'section': rec['section'],
    }

  0%|          | 0/2055 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
answers[2]

In [ ]:
results_flant5 = [None] * len(ground_truth)

for i, value in answers.items():
    results_flant5[i] = value.copy()
    results_flant5[i].update(ground_truth[i])

In [ ]:
df_gpt4o_mini = pd.DataFrame(results_gpt4o_mini)
df_gpt4o_mini.to_csv('/content/resultsflant5.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')